# 02 - Cálculo de Quantis e Classificação de Eventos (EPI, EPN, EPF)

Este notebook realiza:
1. Leitura dos dados mensais do CHIRPS, salvo no notebook anterior.

2. Cálculo dos quantis **95, 45–55 e 5** para cada pixel.

3. Classificação mensal de cada pixel em **EPI, EPN ou EPF**.

4. Salvamento dos resultados como NetCDF para uso posterior.

Classes:
- **EPI (Eventos de Precipitação Intensa):** valores > P95
- **EPN (Eventos de Precipitação Normal):** valores entre P45 e P55
- **EPF (Eventos de Precipitação Fraca):** valores < P5

In [10]:
import xarray as xr
import numpy as np
import os

In [11]:
# Caminho para o NetCDF criado no notebook 01
arquivo_nc = "dataset/netcdf_data/chirps_mensal_1981_2023.nc"

# Diretório de saída para os arquivos classificados
saida_dir = "dataset/netcdf_data"
os.makedirs(saida_dir, exist_ok=True)


In [12]:
# Carregar o arquivo NetCDF

ds = xr.open_dataarray(arquivo_nc)
print(ds)

<xarray.DataArray 'precip' (time: 516, lat: 267, lon: 188)> Size: 104MB
[25901136 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 4kB 1981-01-01 1981-10-01 ... 2023-09-01
  * lat      (lat) float64 2kB -2.75 -2.8 -2.85 -2.9 ... -15.95 -16.0 -16.05
  * lon      (lon) float64 2kB -44.55 -44.5 -44.45 -44.4 ... -35.3 -35.25 -35.2


In [13]:
# Calcular os quantis 95, 45-55 e 5
q95 = ds.quantile(0.95, dim='time')
q45 = ds.quantile(0.45, dim='time')
q55 = ds.quantile(0.55, dim='time')
q5 = ds.quantile(0.05, dim='time')

q95.name, q5.name, q45.name, q55.name

('precip', 'precip', 'precip', 'precip')

In [14]:
# Classificação dos eventos

# EPI: Precipitação acima do percentil 95
epi = ds > q95

# EPN: Precipitação entre percentis 45 e 55
epn = (ds >= q45) & (ds <= q55)

# EPF: Precipitação abaixo do percentil 5
epf = ds < q5

In [15]:
# Checar um pixel aleatório
lat_rand = float(ds.lat[len(ds.lat)//2])
lon_rand = float(ds.lon[len(ds.lon)//2])

print(f"Pixel central: lat={lat_rand}, lon={lon_rand}")
print("Exemplo classificação EPI:", epi.sel(lat=lat_rand, lon=lon_rand).values[:12])  # primeiros 12 meses


Pixel central: lat=-9.400001034140587, lon=-39.84999791160226
Exemplo classificação EPI: [False False False False False  True False False False False False False]


In [16]:
# Salvar os resultados em NetCDF
epi.to_netcdf(os.path.join(saida_dir, "epi.nc"))
epn.to_netcdf(os.path.join(saida_dir, "epn.nc"))
epf.to_netcdf(os.path.join(saida_dir, "epf.nc"))

print("Arquivos salvos em dataset/netcdf_data/: epi.nc, epn.nc, epf.nc")


Arquivos salvos em dataset/netcdf_data/: epi.nc, epn.nc, epf.nc


- Cada arquivo salvo possui dimensões iguais ao dataset original: (time, lat, lon).

- Valores são booleanos (`True` para meses que atendem à condição do evento).

- Próximo passo: usar estes arquivos para calcular frequência e intensidade (notebook 03).
